In [15]:
import pandas as pd 
import numpy as np
import re
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from my_module import * 
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler



In [16]:
df=pd.read_csv('movies_cleaned_str_int.csv')


In [17]:
df.columns, df.shape, df.isnull().sum()

(Index(['Unnamed: 0', 'awards.nominations', 'awards.wins', 'cast', 'countries',
        'directors', 'fullplot', 'genres', 'imdb.rating', 'imdb.votes',
        'languages', 'num_mflix_comments', 'plot', 'rated', 'runtime', 'title',
        'tomatoes.consensus', 'tomatoes.fresh', 'tomatoes.production',
        'tomatoes.rotten', 'writers', 'year'],
       dtype='object'),
 (21319, 22),
 Unnamed: 0             0
 awards.nominations     0
 awards.wins            0
 cast                   0
 countries              0
 directors              0
 fullplot               0
 genres                 0
 imdb.rating            0
 imdb.votes             0
 languages              0
 num_mflix_comments     0
 plot                   0
 rated                  0
 runtime                0
 title                  0
 tomatoes.consensus     0
 tomatoes.fresh         0
 tomatoes.production    0
 tomatoes.rotten        0
 writers                0
 year                   0
 dtype: int64)

In [18]:
df.describe()

,Unnamed: 0,awards.nominations,awards.wins,imdb.rating,imdb.votes,num_mflix_comments,runtime,tomatoes.consensus,tomatoes.fresh,tomatoes.production,tomatoes.rotten,year
count,21319.000000,21319.000000,21319.000000,21319.000000,2.131900e+04,21319.000000,21319.000000,21319.000000,21319.000000,21319.000000,21319.000000,21319.000000
mean,11753.059759,4.940476,4.159154,6.665378,2.160998e+04,1.834842,102.566678,0.237159,18.310052,0.237159,10.176603,1996.202026
std,6777.993043,10.384130,9.364558,0.974831,6.663829e+04,13.972165,18.321184,0.425351,34.841986,0.425351,22.118912,18.835177
min,5.000000,0.000000,0.000000,1.600000,5.000000e+00,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,1913.000000
25%,5869.500000,1.000000,0.000000,6.100000,6.580000e+02,0.000000,90.000000,0.000000,0.000000,0.000000,0.000000,1989.000000
50%,11802.000000,2.000000,2.000000,6.800000,2.283000e+03,0.000000,100.000000,0.000000,3.000000,0.000000,1.000000,2003.000000
75%,17629.500000,5.000000,4.000000,7.300000,1.049650e+04,1.000000,112.000000,0.000000,20.000000,0.000000,8.000000,2010.000000
max,23529.000000,256.000000,267.000000,9.500000,1.521105e+06,161.000000,171.000000,1.000000,312.000000,1.000000,195.000000,2015.000000


In [19]:
# selecting the relevant features for recommendation

selected_features = df[['directors','fullplot','cast','countries', 'genres', 'languages','title' ]]
selected_features

,directors,fullplot,cast,countries,genres,languages,title
0,george loane tucker,not rated,jane gail ethel grandin william h turner ...,usa,crime drama,english,traffic in souls
1,edward s curtis,not rated,stanley hunt sarah constance smith hunt mr...,usa,drama history,english,in the land of the head hunters
2,d w griffith,two brothers phil and ted stoneman visit the...,lillian gish mae marsh henry b walthall ...,usa,drama history romance,not rated,the birth of a nation
3,cecil b demille,edith hardy uses charity funds for wall street...,fannie ward sessue hayakawa jack dean ja...,usa,drama,english,the cheat
4,reginald barker,an immigrant leaves his sweetheart in italy to...,george beban clara willia j frank burke ...,usa,drama,english,the italian
...,...,...,...,...,...,...,...
21314,dietrich brèggemann,the african german writer sebastian klein is o...,oliver baumgarten bernd begemann ruth bick...,germany,comedy,german,heil
21315,chung lee,a gu enlists a group of contract killers in th...,hsiao chuan chang qian wan tang sui yann...,taiwan,comedy,mandarin,the laundryman
21316,marcin koszalka,karol an ordinary young man runs into a much...,filip plawiak adam woronowicz julia kijows...,poland czech republic slovakia,thriller,polish english,the red spider
21317,anne èmond,guy s tragic death is a shock for the leblanc ...,maxim gaudette karelle tremblay valerie ca...,canada,drama,french,our loved ones


In [20]:
# Removing all the unecesarry spaces 

selected_features = selected_features.applymap(lambda x: x.strip().replace("  "," "))

In [21]:
selected_features.isnull().sum()

directors    0
fullplot     0
cast         0
countries    0
genres       0
languages    0
title        0
dtype: int64

In [22]:
year = df[['year']]
frame = [selected_features, year]
df = pd.concat([selected_features, year], axis=1)
df


,directors,fullplot,cast,countries,genres,languages,title,year
0,george loane tucker,not rated,jane gail ethel grandin william h turner ma...,usa,crime drama,english,traffic in souls,1913
1,edward s curtis,not rated,stanley hunt sarah constance smith hunt mrs ...,usa,drama history,english,in the land of the head hunters,1914
2,d w griffith,two brothers phil and ted stoneman visit their...,lillian gish mae marsh henry b walthall mir...,usa,drama history romance,not rated,the birth of a nation,1915
3,cecil b demille,edith hardy uses charity funds for wall street...,fannie ward sessue hayakawa jack dean james...,usa,drama,english,the cheat,1915
4,reginald barker,an immigrant leaves his sweetheart in italy to...,george beban clara willia j frank burke leo...,usa,drama,english,the italian,1915
...,...,...,...,...,...,...,...,...
21314,dietrich brèggemann,the african german writer sebastian klein is o...,oliver baumgarten bernd begemann ruth bickel...,germany,comedy,german,heil,2015
21315,chung lee,a gu enlists a group of contract killers in th...,hsiao chuan chang qian wan tang sui yann ya...,taiwan,comedy,mandarin,the laundryman,2015
21316,marcin koszalka,karol an ordinary young man runs into a much o...,filip plawiak adam woronowicz julia kijowska...,poland czech republic slovakia,thriller,polish english,the red spider,2015
21317,anne èmond,guy s tragic death is a shock for the leblanc ...,maxim gaudette karelle tremblay valerie cadi...,canada,drama,french,our loved ones,2015


In [27]:
# combining all the 7 selected features

combined_features = df['genres']+' '+df['cast']+' '+ df['directors']+' '+ df['fullplot']+' '+ df['genres'] + ' ' + df['countries'] + ' ' + df['languages']
#combined_features = pd.concat([combined_features, year], axis=1)
print(combined_features)

0        crime  drama jane gail  ethel grandin  william...
1        drama  history stanley hunt  sarah constance s...
2        drama  history  romance lillian gish  mae mars...
3        drama fannie ward  sessue hayakawa  jack dean ...
4        drama george beban  clara willia  j frank burk...
                               ...                        
21314    comedy oliver baumgarten  bernd begemann  ruth...
21315    comedy hsiao chuan chang  qian wan  tang sui  ...
21316    thriller filip plawiak  adam woronowicz  julia...
21317    drama maxim gaudette  karelle tremblay  valeri...
21318    comedy  drama  romance holly aird  eileen atki...
Length: 21319, dtype: object


In [34]:
# Initiate the vectorizer and converting the text data to feature vectors

cv = CountVectorizer()

vectorizer = TfidfVectorizer()

feature_vectors = vectorizer.fit_transform(combined_features)

In [31]:
# Find similarity by applying cosine to the vectors previsoulsy made 

similarity = cosine_similarity(feature_vectors)


In [10]:
indices = pd.Series(df.index, index=df['title'])
indices = indices[~indices.index.duplicated(keep='last')]
indices

title
traffic in souls                       0
in the land of the head hunters        1
the birth of a nation                  2
the cheat                              3
civilization                           6
                                   ...  
heil                               21314
the laundryman                     21315
the red spider                     21316
our loved ones                     21317
scenes of a sexual nature          21318
Length: 19718, dtype: int64

In [11]:
target_movie_index = indices['american history x']
target_movie_index

7521

In [12]:
similarity[target_movie_index]


array([0.04654461, 0.04341107, 0.03344956, ..., 0.02501933, 0.03963819,
       0.04057633])

In [13]:
similarity_scores = pd.DataFrame(similarity[target_movie_index], columns=["score"])

In [14]:
movie_indices = similarity_scores.sort_values("score", ascending=False)[0:11].index
movie_indices

Int64Index([7521, 16280, 7175, 7712, 5600, 4003, 10842, 4530, 8830, 1704,
            5805],
           dtype='int64')

In [15]:
df['title'].iloc[movie_indices]

7521      american history x
16280               baseline
7175               the boxer
7712               the boxer
5600             american me
4003     broadway danny rose
10842        danny deckchair
4530               bad taste
8830          animal factory
1704            the ceremony
5805           south central
Name: title, dtype: object

In [17]:
def get_films_by_name(movie_name, movie_indices):
    return movie_indices[movie_indices.index.str.contains(movie_name, na=False)]

get_films_by_name('american history', indices)

title
american history x    7521
dtype: int64

In [18]:

def get_recommended_movies(target_movie_index, movie_similarities,movies_df):
    similarity_scores = pd.DataFrame(movie_similarities[target_movie_index], columns=["score"]) 
    movie_indices = similarity_scores.sort_values("score", ascending=False)[0:11].index 
    return df['title'].iloc[movie_indices]

get_recommended_movies(7528,similarity,df) 

7528              all the little animals
14846                  prayers for bobby
7479                              u turn
17913    bobby fischer against the world
4205                    the heavenly kid
18100                        foreverland
4019                   crimes of passion
8257                   short sharp shock
13416                   we own the night
9301                               bully
7376                        monument ave
Name: title, dtype: object

### Recommandation using built in library Difflib 

In [221]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')

In [222]:
# Making a list of title  

list_of_all_titles = df['title'].tolist()
print(list_of_all_titles[:10])

['traffic in souls', 'in the land of the head hunters', 'the birth of a nation', 'the cheat', 'the italian', 'regeneration', 'civilization', 'hell s hinges', 'where are my children', 'the poor little rich girl']


In [304]:
# finding the close match for the movie name given by the user
# Using diiflib library and the function get_close_matches 
"""
Args: 

    word: a sequence for which close matches are desired (typically a string).
    possibilities: a list of sequences against which to match word (typically a list of strings).
    n: the maximum number of close matches to return. This parameter is set to 3 by default.
And one optional parameter:

It starts by checking the value of n and cutoff to make sure they are valid input.
Then, it initializes an empty list result and creates an instance of SequenceMatcher class and sets the word as the second sequence.

Then it loops through each element in the possibilities list, sets it as the first sequence in the SequenceMatcher instance and compares it to the second sequence (the input word) 
using three methods real_quick_ratio(), quick_ratio(), and ratio(). If all of them are greater or equal to the cutoff value, it appends a tuple of the ratio and the possibility to the result list.

Finally, it uses the _nlargest() function from the heapq module to get the n largest elements from the result list and returns the list of matches sorted by similarity score, most similar first.

It also contains examples of how the function should work, which demonstrates that the function returns a list of the best "good enough" matches based on the input word and the 
possibilities list, using the ratio of the similarity between the input word and the possibilities as the criteria of similarity.

The get_close_matches function is used to find the best "good enough" matches for a given word from a list of possibilities. It uses the SequenceMatcher class from the difflib library
to compare the input word to each possibility in the list and finds the closest matches based on similarity score. The function takes in three mandatory parameters: 
the input word, the list of possibilities and the number of matches to return, and one optional parameter, the cutoff value, which is a float in the range of [0, 1] 
that represents the minimum similarity score required for a possibility to be considered a match. The function returns a list of the best matches, sorted by similarity score, 
most similar first.
"""

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles, n=10)
print(find_close_match)

['svend', 'se7en', 'swerve', 'stevie', 'sheena', 'selena', 'heaven', 'heaven', 'evelyn', 'severance']


In [305]:
close_match = find_close_match[0]
print(close_match)

svend


In [306]:
# finding the index of the movie with title

index_of_the_movie = df[df.title == close_match]['index'].values[0]
print(index_of_the_movie)

18871


In [307]:
# getting a list of similar movies

similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [308]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

In [309]:
# print the name of similar movies based on the index

print('Movies suggested : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<10):
    print(i, '.',title_from_index)
    i+=1

Movies suggested : 

1 . svend
2 . the green butchers
3 . man with a movie camera
4 . above the street  below the water
5 . at kende sandheden
6 . mindwalk
7 . kamikaze taxi
8 . raajneeti
9 . armadillo


In [310]:
# Entire steps to find the recommandation

# Ask the user to write its name
movie_name = input(' Enter your favourite movie name : ')

# Making the list of movies name
list_of_all_titles = df['title'].tolist()

# 
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = df[df.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if (i<10):
    print(i, '.',title_from_index)
    i+=1

Movies suggested : 

1 . american history x
2 . the boxer
3 . the boxer
4 . baseline
5 . broadway danny rose
6 . bad taste
7 . danny deckchair
8 . south central
9 . the ceremony


### Apply numerical values to expect better results

In [226]:
df=pd.read_csv('movies_cleaned_str_int.csv')
# Removing ungly columns  and creating a new column of index to be used later 
df = df.drop(['Unnamed: 0', 'plot'], axis=1)
df['index'] = df.index
selected_features = selected_features.applymap(lambda x: x.strip().replace("  "," "))

In [252]:
numerical_features = df[['awards.nominations', 'runtime', 'awards.wins', 'imdb.rating', 'imdb.votes', 'tomatoes.consensus', 'tomatoes.fresh', 'tomatoes.rotten', 'year']]
df.columns

Index(['awards.nominations', 'awards.wins', 'cast', 'countries', 'directors',
       'fullplot', 'genres', 'imdb.rating', 'imdb.votes', 'languages',
       'num_mflix_comments', 'rated', 'runtime', 'title', 'tomatoes.consensus',
       'tomatoes.fresh', 'tomatoes.production', 'tomatoes.rotten', 'writers',
       'year', 'index'],
      dtype='object')

In [253]:
numerical_features.describe()

,awards.nominations,runtime,awards.wins,imdb.rating,imdb.votes,tomatoes.consensus,tomatoes.fresh,tomatoes.rotten,year
count,21377.000000,21377.000000,21377.000000,21377.000000,2.137700e+04,21377.000000,21377.000000,21377.000000,21377.000000
mean,4.937971,102.560556,4.161622,6.665566,2.157271e+04,0.236797,18.290359,10.160125,1996.193713
std,10.377024,18.321462,9.361993,0.974736,6.656715e+04,0.425126,34.824631,22.097117,18.828899
min,0.000000,40.000000,0.000000,1.600000,5.000000e+00,0.000000,0.000000,0.000000,1913.000000
25%,1.000000,90.000000,0.000000,6.100000,6.560000e+02,0.000000,0.000000,0.000000,1989.000000
50%,2.000000,100.000000,2.000000,6.800000,2.277000e+03,0.000000,3.000000,1.000000,2003.000000
75%,5.000000,112.000000,4.000000,7.300000,1.047100e+04,0.000000,20.000000,8.000000,2010.000000
max,256.000000,171.000000,267.000000,9.500000,1.521105e+06,1.000000,312.000000,195.000000,2015.000000


In [254]:
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(numerical_features), columns=numerical_features.columns)

In [259]:
df_scaled

,awards.nominations,runtime,awards.wins,imdb.rating,imdb.votes,tomatoes.consensus,tomatoes.fresh,tomatoes.rotten,year
0,0.000000,0.366412,0.003745,0.556962,0.000241,0.0,0.000000,0.000000,0.000000
1,0.000000,0.190840,0.003745,0.531646,0.000143,0.0,0.000000,0.000000,0.009804
2,0.000000,0.954198,0.007491,0.658228,0.010328,1.0,0.121795,0.000000,0.019608
3,0.000000,0.145038,0.003745,0.620253,0.001088,0.0,0.000000,0.000000,0.019608
4,0.000000,0.290076,0.003745,0.607595,0.000112,0.0,0.000000,0.000000,0.019608
...,...,...,...,...,...,...,...,...,...
21372,0.003906,0.473282,0.000000,0.468354,0.000090,0.0,0.000000,0.000000,1.000000
21373,0.007812,0.534351,0.000000,0.594937,0.000014,0.0,0.000000,0.000000,1.000000
21374,0.003906,0.381679,0.000000,0.734177,0.000007,0.0,0.000000,0.000000,1.000000
21375,0.003906,0.473282,0.000000,0.734177,0.000000,0.0,0.000000,0.000000,1.000000


In [256]:
combined_features 

0        crime jane gail   ethel grandin   william h  t...
1        drama stanley hunt   sarah constance smith hun...
2        drama lillian gish   mae marsh   henry b  walt...
3        drama fannie ward   sessue hayakawa   jack dea...
4        drama george beban   clara willia    j  frank ...
                               ...                        
21372    comedy oliver baumgarten   bernd begemann   ru...
21373    comedy hsiao chuan chang   qian wan   tang sui...
21374    thriller filip plawiak   adam woronowicz   jul...
21375    drama maxim gaudette   karelle tremblay   vale...
21376    comedy holly aird   eileen atkins   hugh bonne...
Length: 21377, dtype: object